# Security

* This notebook uses Application Default Credentials (ADC) for BigQuery authentication.
* Authenticate with gcloud before running: `gcloud auth application-default login`
* No API key files needed - authentication is handled through your gcloud session.

# Tools

In [ ]:
# Package and library installation
packages_needed = c("tidyverse", "knitr") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Using Application Default Credentials - no manual key file needed if you've run `gcloud auth application-default login`

In [ ]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘rapidjsonr’




# Source

## Database Connection

In [ ]:
# BigQuery Authentication using Application Default Credentials
# This will use your gcloud authentication automatically
# Run this first in terminal: gcloud auth application-default login
bq_auth()

In [ ]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [ ]:
billing <- bq_test_project()

## Database Query

### gridVeg_plant_intercepts

In [ ]:
sql_plant_intercepts <-
"
  SELECT
    survey_ID,
    grid_point,
    key_plant_code,
    plant_native_status,
    plant_life_cycle,
    plant_life_form,
    intercepts_pct
  FROM
    `mpg-data-warehouse.vegetation_gridVeg_summaries.gridVeg_foliar_cover_all`
"
bq_plant_intercepts <- bq_project_query(billing, sql_plant_intercepts)
tb_plant_intercepts <- bq_table_download(bq_plant_intercepts)
df_plant_intercepts <- as.data.frame(tb_plant_intercepts) %>% glimpse()

Rows: 26,387
Columns: 7
$ survey_ID           <chr> "436", "436", "436", "436", "436", "436", "436", "…
$ grid_point          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ key_plant_code      <chr> "ACHMIL", "SEDLAN", "ELYTRA", "VALDIO", "GALAPA", …
$ plant_native_status <chr> "native", "native", "native", "native", "native", …
$ plant_life_cycle    <chr> "perennial", "perennial", "perennial", "perennial"…
$ plant_life_form     <chr> "forb", "forb", "graminoid", "forb", "forb", "forb…
$ intercepts_pct      <dbl> 1.5, 0.5, 2.5, 1.0, 1.5, 0.5, 1.0, 1.0, 2.5, 2.0, …


### gridVeg_survey_metadata

In [ ]:
sql_survey_metadata <- "
  SELECT
    survey_ID,
    year,
    survey_sequence,
    grid_point
  FROM
    mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata
"
bq_survey_metadata <- bq_project_query(billing, sql_survey_metadata)
tb_survey_metadata <- bq_table_download(bq_survey_metadata)
df_survey_metadata <- as.data.frame(tb_survey_metadata) %>% glimpse()

Rows: 1,615
Columns: 4
$ survey_ID       <chr> "138", "139", "135", "134", "137", "136", "141", "143"…
$ year            <int> 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, …
$ survey_sequence <chr> "2010", "2010", "2010", "2010", "2010", "2010", "2010"…
$ grid_point      <int> 61, 60, 52, 51, 62, 44, 57, 66, 65, 59, 69, 76, 75, 68…


# Wrangle

Start with the view vegetation_gridVeg_summaries:gridVeg_plant_intercepts. Remove records where key_plant_code = “NV” (corresponds with key_plant_species = 360).

## Explore key_plant_code "NV"
No records should exist with no vegetation (NV) because this is built with the gridVeg_foliar_cover_all data.

In [ ]:
df_plant_intercepts %>%
  filter(key_plant_code == "NV") %>%
  glimpse()

Rows: 0
Columns: 7
$ survey_ID           <chr> 
$ grid_point          <int> 
$ key_plant_code      <chr> 
$ plant_native_status <chr> 
$ plant_life_cycle    <chr> 
$ plant_life_form     <chr> 
$ intercepts_pct      <dbl> 


In [ ]:
df_plant_functional_groups <- df_plant_intercepts %>%
  filter(key_plant_code != "NV")

## Calculate Detection Rate

Group the data on {survey_ID, plant_native_status, plant_life_cycle, plant_life_form}. Sum the intercepts_pct within each group.

In [ ]:
df_plant_functional_groups %>%
  group_by(survey_ID, plant_native_status, plant_life_cycle, plant_life_form) %>%
  summarise(detection_rate = sum(intercepts_pct)) %>% ungroup() %>%
  filter(survey_ID == "436")

`summarise()` has grouped output by 'survey_ID', 'plant_native_status',
'plant_life_cycle'. You can override using the `.groups` argument.


survey_ID,plant_native_status,plant_life_cycle,plant_life_form,detection_rate
<chr>,<chr>,<chr>,<chr>,<dbl>
436,native,annual,forb,3.5
436,native,perennial,forb,9.5
436,native,perennial,graminoid,44.5
436,native,perennial,shrub,10.5
436,native,perennial,tree,2.0
436,native,unknown,forb,0.5
436,nonnative,annual,forb,1.5
436,nonnative,annual,graminoid,5.5
436,nonnative,perennial,forb,1.5


In [ ]:
df_detection_rate <- df_plant_functional_groups %>%
  group_by(survey_ID, plant_native_status, plant_life_cycle, plant_life_form) %>%
  summarise(detection_rate = sum(intercepts_pct)) %>%
  ungroup()

`summarise()` has grouped output by 'survey_ID', 'plant_native_status',
'plant_life_cycle'. You can override using the `.groups` argument.


In [ ]:
df_detection_rate %>% glimpse()

Rows: 9,644
Columns: 5
$ survey_ID           <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5FAD-…
$ plant_native_status <chr> "native", "native", "native", "native", "nonnative…
$ plant_life_cycle    <chr> "annual", "perennial", "perennial", "perennial", "…
$ plant_life_form     <chr> "forb", "forb", "graminoid", "shrub", "forb", "gra…
$ detection_rate      <dbl> 6.5, 15.5, 5.0, 8.5, 5.0, 46.0, 13.5, 26.5, 10.5, …


Then, make sure all combinations of functional groups that are found in the data are represented in each survey_ID. For those groups which were not detected at a survey_ID, fill the detection_rate with 0. This will complete the data set and make averages and other statistical comparisons more meaningful.

The number of records produced in the final dataset should be predictable from the number of surveys and the possible combinations of plant functional groups (pfg). With 1242 surveys and 23 pfg, we should end up with 28,556 records in the final data set.

In [ ]:
# Number of surveys
df_detection_rate$survey_ID %>%
  unique() %>%
  length()

[1] 1385

In [ ]:
# Possible combinations of pfg
df_detection_rate %>%
  select(plant_native_status, plant_life_cycle, plant_life_form) %>%
  distinct() %>%
  arrange(plant_native_status, plant_life_cycle)

plant_native_status,plant_life_cycle,plant_life_form
<chr>,<chr>,<chr>
native,annual,forb
native,annual,graminoid
native,biennial,forb
native,perennial,forb
native,perennial,graminoid
native,perennial,shrub
native,perennial,tree
native,perennial,vine
native,unknown,forb


## Group Fill

In [ ]:
df_plant_functional_groups_comp <-
  df_detection_rate %>% ungroup() %>%
    complete(survey_ID,
            nesting(plant_native_status, plant_life_cycle, plant_life_form),
            fill = list(detection_rate = 0)) %>%
    glimpse()

Rows: 31,855
Columns: 5
$ survey_ID           <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5FAD-…
$ plant_native_status <chr> "native", "native", "native", "native", "native", …
$ plant_life_cycle    <chr> "annual", "annual", "biennial", "perennial", "pere…
$ plant_life_form     <chr> "forb", "graminoid", "forb", "forb", "graminoid", …
$ detection_rate      <dbl> 6.5, 0.0, 0.0, 15.5, 5.0, 8.5, 0.0, 0.0, 0.0, 5.0,…


## Join Metadata

Finally, join the metadata from requested tables or views and complete the schema below.

In [ ]:
# bring gridpoints back in
grid_point_ref <- df_plant_intercepts %>%
  select(survey_ID, grid_point) %>%
  distinct(survey_ID, grid_point)

In [ ]:
df_plant_functional_groups_join <-
  df_plant_functional_groups_comp %>%
  left_join(df_survey_metadata, by = "survey_ID") %>%
  select(survey_ID, year, survey_sequence, grid_point, everything()) %>%
  arrange(year, grid_point) %>%
  glimpse()

Rows: 31,855
Columns: 8
$ survey_ID           <chr> "436", "436", "436", "436", "436", "436", "436", "…
$ year                <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 20…
$ survey_sequence     <chr> "2011-12", "2011-12", "2011-12", "2011-12", "2011-…
$ grid_point          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ plant_native_status <chr> "native", "native", "native", "native", "native", …
$ plant_life_cycle    <chr> "annual", "annual", "biennial", "perennial", "pere…
$ plant_life_form     <chr> "forb", "graminoid", "forb", "forb", "graminoid", …
$ detection_rate      <dbl> 3.5, 0.0, 0.0, 9.5, 44.5, 10.5, 2.0, 0.0, 0.5, 1.5…


In [ ]:
df_plant_functional_groups_join %>% distinct(year)

year
<int>
2011
2012
2013
2015
2016
2017
2021
2022


## Filter

In [ ]:
df_plant_functional_groups_output <- df_plant_functional_groups_join %>%
  filter(year > 2022)

# Output

In [ ]:
# updated output 2022-08-18 esamsoe
# write_csv(df_plant_functional_groups_output, file = "gridVeg_plant_functional_groups-WRANGLE.csv")